In [1]:
load_model = 'convnext_large_based_model2.pt'

data_dir = '../../../stanford_dogs_new/'

In [2]:
# import numpy as np

from IPython.display import Image

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

from torchvision.datasets import ImageFolder
from PIL import Image

from torch.autograd import Variable

from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


from IPython.display import Image
from IPython.display import Markdown

from PIL import Image
import PIL.Image

import pandas as pd

# sklearn metrics
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import f1_score


## load the convnext_large based model

In [3]:
convnext_large_loaded = models.convnext_large(pretrained=True)

for param in convnext_large_loaded.parameters():
    param.requires_grad = False

    
# densenet121_loaded.classifier = nn.Sequential(*list(model_densenet121.classifier.children())[:-1] + [nn.Linear(in_features=1024, out_features=120, bias=True)])
convnext_large_loaded.classifier[2] = nn.Linear(in_features=1536, out_features=120, bias=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
convnext_large_loaded = convnext_large_loaded.to(device)



convnext_large_loaded.load_state_dict(torch.load('convnext_large_based_model2.pt'), strict=False)
convnext_large_loaded.eval()

ConvNeXt(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((192,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
          (1): Permute()
          (2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=192, out_features=768, bias=True)
          (4): GELU()
          (5): Linear(in_features=768, out_features=192, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
          (1): Permute()
          (2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=192, out

In [4]:
model = convnext_large_loaded

In [5]:
# preprocess the image for the model
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
])

In [6]:
path = '../../../stanford_dogs_new/test'

list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]

# gets the breed name from subfolders name like 'n02099429-curly-coated_retriever'
def rename(name):
    return ' '.join(' '.join(name.split('-')[1:]).split('_'))

per_class_accuracy = dict()
total_accuracy = 0

y_true = []
y_pred = []

for breed_dir in list_subfolders_with_paths:
    
    breed = rename(breed_dir.split('/')[-1])
    
    images = os.listdir(breed_dir)
    
    matched = False
    matches = 0
    total_images = len(images)

    for image in images:

        input_image = Image.open(breed_dir + '/' + image)


        input_tensor = preprocess(input_image)
        if torch.cuda.is_available():
            input_tensor = Variable(input_tensor.cuda())

        input_batch = input_tensor.unsqueeze(0)
        out = model(input_batch)

        probabilities = torch.nn.functional.softmax(out[0], dim=0)
        # print(probabilities)

        with open("../../stanford_dogs_breeds_classes_final.txt", "r") as f:
            categories = [s.strip() for s in f.readlines()]

        predicted_breeds = []
        top1_prob, top1_catid = torch.topk(probabilities, 1)
        for i in range(top1_prob.size(0)):
            # predicted_breeds.append([categories[top3_catid[i]], top3_prob[i].item()*100])
            predicted_breed = categories[top1_catid[i]]
            y_true.append(breed)
            y_pred.append(predicted_breed)

        # list to be used directly by the application (predicted_breed, probability)
        # print("predicted_breeds are: \n", predicted_breed)
        if breed == predicted_breed:
            matched = True
            matches = matches + 1

        per_class_accuracy[breed] = matches / total_images * 100

In [7]:
df = pd.DataFrame(list(zip(y_true, y_pred)),
               columns =['Actual_breed', 'Predicted_breed'])

df

,Actual_breed,Predicted_breed
0,Pomeranian,Pomeranian
1,Pomeranian,Pomeranian
2,Pomeranian,Pomeranian
3,Pomeranian,Pomeranian
4,Pomeranian,Pomeranian
...,...,...
2105,papillon,papillon
2106,papillon,papillon
2107,papillon,papillon
2108,papillon,papillon


## Confusion Matrix

In [8]:
# Print the confusion matrix
print(metrics.confusion_matrix(y_true, y_pred))


[[24  0  0 ...  0  0  0]
 [ 0 17  0 ...  0  0  0]
 [ 0  0 21 ...  0  0  0]
 ...
 [ 0  0  0 ... 16  0  0]
 [ 0  0  0 ...  0 19  0]
 [ 0  0  0 ...  0  0 16]]


## Precision, Recall and F1

In [9]:
# Print the precision and recall, among other metrics
print(metrics.classification_report(y_true, y_pred, digits=3))

                                precision    recall  f1-score   support

                  Afghan hound      1.000     1.000     1.000        24
           African hunting dog      0.944     1.000     0.971        17
                      Airedale      1.000     1.000     1.000        21
American Staffordshire terrier      0.938     0.882     0.909        17
                   Appenzeller      0.933     0.875     0.903        16
            Australian terrier      1.000     0.900     0.947        20
            Bedlington terrier      1.000     1.000     1.000        19
          Bernese mountain dog      0.917     1.000     0.957        22
              Blenheim spaniel      1.000     1.000     1.000        19
                 Border collie      0.812     0.867     0.839        15
                Border terrier      1.000     1.000     1.000        18
                   Boston bull      0.905     1.000     0.950        19
          Bouvier des Flandres      1.000     0.933     0.966  

## Macro F1 Score

In [10]:
from sklearn.metrics import f1_score

f1_score(y_true, y_pred, average='macro')


0.9672235190883598

## Micro F1 Score

In [11]:
f1_score(y_true, y_pred, average='micro')


0.9687203791469194

## Weighted F1 Score

In [12]:
f1_score(y_true, y_pred, average='weighted')

0.9683416978251931

## Cohen Kappa Score

In [13]:
from sklearn.metrics import cohen_kappa_score

cohen_kappa_score(y_true, y_pred)

0.9684527651176293

## Total Accuracy calculated from per class accuracy

In [14]:
# calculated from per class accuracy
total_accuracy = sum(per_class_accuracy.values())/120
total_accuracy

96.747123640893

## Per Breed Accuracy

### Top 10 Breeds with least accuracy

In [15]:
k = 10
sorted(per_class_accuracy.items(), key=lambda x: x[1])[:k]

[('Eskimo dog', 46.666666666666664),
 ('standard schnauzer', 81.25),
 ('collie', 81.25),
 ('Norwich terrier', 84.21052631578947),
 ('dhole', 86.66666666666667),
 ('Border collie', 86.66666666666667),
 ('curly coated retriever', 87.5),
 ('boxer', 87.5),
 ('Appenzeller', 87.5),
 ('Sussex spaniel', 87.5)]

### Top 10 Breeds with most accuracy

In [16]:
k = 10
sorted(per_class_accuracy.items(), key=lambda x: x[1], reverse=True)[:k]

[('Pomeranian', 100.0),
 ('basset', 100.0),
 ('Japanese spaniel', 100.0),
 ('Brabancon griffon', 100.0),
 ('Blenheim spaniel', 100.0),
 ('Shih Tzu', 100.0),
 ('Irish setter', 100.0),
 ('German short haired pointer', 100.0),
 ('flat coated retriever', 100.0),
 ('Irish water spaniel', 100.0)]